# Overview

The purpose of this notebook is to create a weekly returns CSV file for all the funds in our portfolio that can be easily used in our multi-armed bandit

# Setup

## Import Libraries

In [29]:
import pandas as pd
import numpy as np
from datetime import datetime, timedelta

## Defined Variables

In [31]:
#CRSP characteristics for the selected funds
fund_characteristics = {5887: {'full':'BlackRock Funds III: iShares S&P 500 Index Fund; Class K Shares',
                       'short':'iShares S&P 500 Index Fund',
                       'class':'K'},
                16431: {'full':'iShares Trust: iShares 7-10 Year Treasury Bond ETF',
                       'short':'iShares 7-10 Year Treasury Bond ETF',
                       'class':'ETF'}}

#List of CSV filenames
filenames = ['crsp_iShares_S&P500_Index_Fund_daily_ret.csv',
             'crsp_iShares_7-10_Year_Treasury_Bond_ETF_daily_ret.csv']

#File Path
filepath = 'PATH_TO_DATA'

#Functions

## Convert Between Daily and Weekly Returns Helper Function

In [32]:
def daily_to_weekly_returns(daily_returns):
  
  #Resample daily returns to weekly returns using Fridays as the weekly anchor
  weekly_returns = daily_returns.resample('W-FRI', on='caldt').sum().reset_index()

  #Rename metrics from daily to weekly
  weekly_returns.rename(columns={'dnav': 'weekly_nav','dret': 'weekly_return'},
                        inplace=True)
  #Map weekly net asset value for each fund
  weekly_returns['weekly_nav'] = weekly_returns['caldt'].map(daily_returns.set_index('caldt')['dnav'])

   # Re-map the fund number for each entry
  weekly_returns['crsp_fundno'] = daily_returns['crsp_fundno']

    #If there is no data for a given Friday, due a holiday (no trading), use the previous day's metrics
  for na_date in weekly_returns[weekly_returns.isnull().any(axis=1)]['caldt']:
    weekly_returns.loc[weekly_returns['caldt'] == na_date, 'weekly_nav'] = daily_returns[daily_returns['caldt'] == na_date - timedelta(days=1)]['dnav'].values[0]


  return weekly_returns

## Create Weekly Return Dataframe

In [33]:
def get_weekly_ret():
  #Create empty weekly return dataframe
  weekly_ret = pd.DataFrame(columns=['crsp_fundno',	'caldt',	'weekly_nav',	'weekly_return'])

  #Read in CSV file, convert daily returns to weekly returns, append to dataframe.
  csv_data = pd.read_csv(filepath + 'CRSP_SelectFund_Daily_Returns.csv')
  csv_data['caldt'] =  pd.to_datetime(csv_data['caldt'], format='%m/%d/%Y')

  #Get weekly returns for each CRSP fund
  for crsp_fundno in fund_characteristics:
    fund_daily_returns = csv_data[csv_data['crsp_fundno'] == crsp_fundno].reset_index()

    weekly_ret = weekly_ret.append(daily_to_weekly_returns(fund_daily_returns), ignore_index=True)

    #Set the fund name column for each fund
    weekly_ret.loc[weekly_ret['crsp_fundno'] == crsp_fundno, 'fund_name'] = fund_characteristics[crsp_fundno]['short']

  #Pivot the dataframe to have the date as the index,  fund names as columns and the weekly returns as values
  weekly_ret = weekly_ret.pivot(index='caldt', columns='fund_name', values='weekly_return')

  return weekly_ret

In [34]:
weekly_returns = get_weekly_ret()

weekly_returns

fund_name,iShares 7-10 Year Treasury Bond ETF,iShares S&P 500 Index Fund
caldt,,
2010-01-08,0.004115,0.027120
2010-01-15,0.010964,-0.007544
2010-01-22,0.006574,-0.038933
2010-01-29,0.000288,-0.016344
2010-02-05,0.006329,-0.006246
...,...,...
2021-12-03,0.009776,-0.011349
2021-12-10,-0.008610,0.038057
2021-12-17,0.006556,-0.019018


## Write Weekly Returns Dataframe to CSV

In [35]:
weekly_returns.to_csv(filepath + 'selectedFundWeeklyReturns.csv', sep='\t', encoding='utf-8', index=True)